# Notebook 2: Sieci neuronowe w TensorFlow
*** 
Ćwiczenia w tym Notebooku dotyczą już tylko sieci neuronowych. Na początek zobaczysz jak
tworzy się sieci neuronowe w TensorFlow.
Następnie utworzysz zestaw funkcji, które posłużą Ci do stworzenia głębokiej sieci
neuronowej, do rozpoznawania cyfr pisma ręcznego.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

## 1. Sieć neuronowa z jedną warstwą ukrytą

Do tego modelu ponownie użyjemy zbioru z poprzedniego Notebooka, lecz tym razem będziemy rozpoznawać wszystkie 
10 cyfr dzięki funkcji SOFTMAX w warstwie wyjściowej.

In [ ]:
#Import danych
X,y = load_digits(n_class=10, return_X_y=True)

np.random.seed(54321)
p = np.random.permutation(len(y))
X = X[p]
y = y[p].reshape(-1,1)

test_set_size = round(len(y) * 0.25)
X_train = X[:-test_set_size]
y_train = y[:-test_set_size]

X_test = X[-test_set_size:]
y_test = y[-test_set_size:]

print("Wymiary X:", X_train.shape)
print("Wymiary y:", y_train.shape)

##### Ćwiczenie 1.1
1. Ustaw hiperparametry modelu - liczbę neurnonów w warstwie ukrytej na 100, learning_rate na 0.01, liczbe epok na 3000.
2. Zdefiniuj placeholdery: X_input:typ - tf.float32 o odpowiednim kształcie i name = 'x_input'; y_target: typ tf.int32, o odpowiednim kształcie i name = 'y_target'.
3. Oblicz z1 wg wzoru $Z_1 = X * W_1 + b_1$ a następnie zaaplikuj sigmoidalną funkcję aktywacji (tf.nn.sigmoid()).
4. Zainicjalizuj W2 i b2, nazwy weight_2 i bias_2, odpowiednio dobierając wymiary, wzoruj się na poprzedniej warstwie.
5. Oblicz z2 wg wzoru $Z_2 = A_1 * W_2 + b_2$
5. W sesji, w kolejnej komórce wywołaj optimizer i dostarcz do grafu odpowiednie dane (feed_dict).
6. Wytrenuj sieć.
7. Zobacz jak zmiana hiperparametrów wpływa na jej rezultaty.

In [ ]:
tf.reset_default_graph()

#hiperparametry modelu
hidden_layer_neurons = ###Miejsce na Twój kod
learning_rate = ###Miejsce na Twój kod
n_epochs = ###Miejsce na Twój kod

#wymiary zmiennych wejściowych
n_outputs = len(np.unique(y_train))
n_inputs = X_train.shape[1]

X_input = ###Miejsce na Twój kod
y_target = ###Miejsce na Twój kod

#kodowanie one hot zmiennej celu
y_onehot = tf.reshape(tf.one_hot(y_target, depth=n_outputs, axis=1), (-1,n_outputs))

#Pierwsza warstwa
initialization_1 = tf.truncated_normal((n_inputs, hidden_layer_neurons), stddev=0.01, seed=54321)
W1 = tf.get_variable(name="weights_1", initializer=initialization_1)
b1 = tf.get_variable(name='bias_1', initializer=tf.zeros([hidden_layer_neurons]))
z1 = ###Miejsce na Twój kod
a1 =###Miejsce na Twój kod


#Druga warstwa - wyjściowa
initialization_2 = tf.truncated_normal((###Miejsce na Twój kod), stddev=0.01, seed=54321)
W2 = ###Miejsce na Twój kod
b2 = ###Miejsce na Twój kod
z2 = ###Miejsce na Twój kod

y_predictions = tf.nn.softmax(z2)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=z2, labels=y_onehot))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_onehot, 1), tf.argmax(y_predictions,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        ###Miejsce na Twój kod
        if epoch % 200 == 0:
            epoch_loss = sess.run(loss, feed_dict={X_input: X_train, y_target: y_train})
            epoch_accuracy = sess.run(accuracy,feed_dict={X_input: X_train, y_target: y_train})
            print("Epoch:{}, loss:{}, accuracy:{}".format(epoch, epoch_loss, epoch_accuracy))
    
    print("\nFINAL METRICS:")
    print("Training set accuracy: ", accuracy.eval({X_input: X_train, y_target: y_train}))
    print("Test set accuracy: ", accuracy.eval({X_input:X_test, y_target: y_test}))

## 2. Dodatkowe elementy  sieci neuronowej
***
Od tego ćwiczenia zaczynamy tworzyć pomocnicze funkcje do modelu głębokiej sieci neuronowej.
W tym zadaniu uzupełnianie kodu będzie wiązało się z większą samodzielnością, warto zatem przy tworzeniu modelu 
podpatrywać rozwiązania z poprzednich przykładów.


### 2.1 Różne funkcje aktywacji

##### Ćwiczenie 2.1
Tworzymy funkcje pomocniczą dodającą warstwę do modelu sieci neuronowej.
1. Zainicjalizuj zmienne W i b, oraz oblicz z, zwróć szególną uwagę na wymiary, wzoruj się na poprzednim przykładzie
2. Nałóż na warstwę odpowiednia funkcję aktywacji w zależności od parametru 'activation' (tf.nn.sigmoid(), tf.nn.relu())

In [ ]:
def ann_layer(X, n_units, name, activation='relu'):
    with tf.variable_scope(name):
        n_inputs = int(X.get_shape()[1])
        initialization = tf.truncated_normal((n_inputs, n_units), stddev=0.01, seed=54321)
        W = ###Miejsce na Twój kod
        b = ###Miejsce na Twój kod
        z = ###Miejsce na Twój kod
        if activation == 'relu':
            return ###Miejsce na Twój kod
        elif activation == 'sigmoid':
            return ###Miejsce na Twój kod
        else:
            return (z, W)

###  2.2 Regularyzacja L2

##### Ćwiczenie 2.2
1. Wykorzystaj wagi i tf.nn.l2_loss() do stworzenia l2_regularizer
2. Utwórz funkcje zwaracającą funkcje straty z nałożoną regularyzacją L2. Działaj wg uproszczonego wzoru $1/m(\sum (loss + \lambda * l2regularizer))$   wykorzystaj tf.reduce_mean()

In [ ]:
def add_l2_regularization(loss, weights, lambd):
    l2_regularizer = ###Miejsce na Twój kod
    return ###Miejsce na Twój kod
    

Sprawdź poprawność funkcji z pomocą dwóch poniższych komórek.

In [ ]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=(None, 2), name="input")
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="target")

W = tf.get_variable("Weight", dtype=tf.float32, initializer=tf.random_normal((2,1),seed=321))
b = tf.get_variable("Bias", initializer=tf.zeros((1,1)))

#Model
model = tf.matmul(X, W) + b

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,labels=y))
regularized_loss = add_l2_regularization(loss, W, 0.2)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(regularized_loss, feed_dict={X: [[5,5],[4,5],[1,1],[3,2]], y:[[1],[1],[0],[0]]}))

Wynik: 0.19133858

### 2.3 Dropout

In [ ]:
def dropout_layer(X, keep_prob=1.0):
    return tf.nn.dropout(X, keep_prob=keep_prob)

## 3. Głęboka sieć neuronowa

##### Ćwiczenie 3.1 
Wykorzystaj utworzone wcześniej funkcje do stworzenia funkcji z modelem sieci neuronowej z 3 warstawmi ukrytymi, pamiętaj o odpowiednich argumentach funkcji

In [ ]:
def neural_net(X, y, n_outputs, input_size=64,
               hidden_layer_1_neurons=100,
               hidden_layer_2_neurons=50, 
               hidden_layer_3_neurons=50,
               dropout_keep_prob = 1.0,
               activation='relu'):
    
    hidden_layer_1 = ###Miejsce na Twój kod
    dropout_1 = ###Miejsce na Twój kod
    hidden_layer_2 =###Miejsce na Twój kod
    dropout_2 = ###Miejsce na Twój kod
    hidden_layer_3 = ###Miejsce na Twój kod
    
    # parametr activation = None, pamiętj o odpowiednich wymiarach
    logits, W = ###Miejsce na Twój kod  
    
    # nałóż na logits funkcje aktywacji SOFTMAX  (tf.nn.softmax())
    y_predictions = ###Miejsce na Twój kod
    # oblicz funkcje straty, użyj tf.reduce_mean() i tf.nn.softmax_cross_entropy_with_logits_v2()
    # Dobierz odpowiednio logits i labels
    loss = ###Miejsce na Twój kod

    return y_predictions, loss, W

##### Ćwiczenie 3.2 
Analogicznie do algorytmu spadku gradientu zaimplementuj algortym Adam

In [ ]:
def optimize(loss, learning_rate=0.001, optimizer='GD'):
    if optimizer == 'GD':
        return tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    elif optimizer == "Adam":
        return ###Miejsce na Twój kod

In [ ]:
def compute_accuracy(y, y_predictions):
    """Funkcja do obliczania dokładności modelu"""
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_predictions,1))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

##### Ćwiczenie 3.3
Korzystając z wcześniej utworzonych funkcji zaimplementuj funkcję do trenowania i ewaluacji modeli

In [ ]:
def model(X_train, y_train, X_test, y_test,
          hidden_layer_1_neurons=100,
          hidden_layer_2_neurons=50,
          hidden_layer_3_neurons=50,
          learning_rate=0.07,
          activation='relu',
          n_epochs=1000,
          l2_regularization=True,
          l2_lambda=0.1,
          dropout_keep_prob=1.0,
          optimizer='GD',
          plot_losses=True):
    
    losses = []
    tf.reset_default_graph()
    
    n_outputs = len(np.unique(y_train))
    n_inputs = X_train.shape[1]
    
    X_input = ##Miejsce na placeholder typ tf.float32
    y_target = ##Miejsce na placeholder typ tf.int32 
    keep_prob = ##Miejsce na placeholder typ tf.float32, shape=()
    
    y_onehot = tf.reshape(tf.one_hot(y_target, depth=n_outputs, axis=1), (-1, n_outputs))
    y_predictions, loss, W = ## Miejsce na model sieci neuronowej o odpowiednich parametrach
    
    if l2_regularization:
        loss = ##Miejsce na regularyzację
    
    optimizer = ##Miejsce na optymalizator
    init = tf.global_variables_initializer()
    
    accuracy = compute_accuracy(y_onehot, y_predictions)
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(n_epochs):
            ###Miejsce na trening modelu / keep_prob=dropout_keep_prob
            if  epoch % 50 == 0:
                epoch_loss = sess.run(loss, feed_dict={X_input: X_train, y_target: y_train, keep_prob: 1.0})
                epoch_accuracy = sess.run(accuracy,feed_dict={X_input: X_train, y_target: y_train, keep_prob: 1.0})
                print("Epoch:{}, loss:{}, accuracy:{}".format(epoch, epoch_loss, epoch_accuracy))
                losses.append(epoch_loss)
                
        if plot_losses:
            plt.plot(losses)
            plt.show()
        
        print("Training set accuracy: ", accuracy.eval({X_input: X_train, y_target: y_train, keep_prob: 1.0}))
        print("Test set accuracy: ", accuracy.eval({X_input:X_test, y_target: y_test, keep_prob: 1.0}))

##### Ćwiczenie 3.4
Wytrenuj i oceń model na zbiorze danych z poprzedniego notebooka.
Eksperymentuj z parametrami sieci tak by uzyskać jak najlepszy wynik na zbiorze testowym

In [ ]:
model(X_train, y_train, X_test, y_test,
          hidden_layer_1_neurons=10,
          hidden_layer_2_neurons=5,
          hidden_layer_3_neurons=6,
          learning_rate=0.07,
          activation='relu',
          n_epochs=1400,
          l2_regularization=True,
          l2_lambda=0.01,
          dropout_keep_prob=0.7)

## 4. MNIST data set
***
Czas na zaaplikowanie stworzonego przez Ciebie modelu
do słynnego w świecie uczenia maszynowego zbioru MNIST.
Zawiera on obrazki pisma ręcznego o wymiarach 28 x 28 pixeli. 50000 przykładów w zbiorze treninigowym i 
10000 w zbiorze testowym. Jest to popularny zbiór do testowania rozwiązań z zakresu rozpoznawania obrazów.
Twoim zdaniem będzie jak najlepsze dobranie parametrów sieci. Nie bój się eksperymentować!
Uwaga - zależnie od komputera, czas nauki sieci może potrwać kilka/kilkanaście minut.

##### Przygotowanie danych

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
mnist_reshape = lambda x: x.reshape(-1, x.shape[1] * x.shape[2])
x_train = mnist_reshape(x_train)
y_train = y_train.reshape(-1,1)
x_test = mnist_reshape(x_test)
y_test = y_test.reshape(-1,1)

##### Modelowanie


In [ ]:
model(x_train, y_train, x_test, y_test,
          hidden_layer_1_neurons=40,
          hidden_layer_2_neurons=15,
          hidden_layer_3_neurons=15,
          learning_rate=0.01,
          activation='relu',
          n_epochs=200,
          l2_regularization=False,
          l2_lambda=0.01,
          dropout_keep_prob=0.6, 
          optimizer='Adam')